In [1]:
# Import libraries.
import pandas as pd, numpy as np, os
from pandas import Timestamp
from datetime import date, datetime 
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from Gantt_plot import *
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
xlsm = "Work Schedule.xlsm"

In [3]:
# All sheetnames in workbook.
wb = load_workbook(xlsm, read_only=True, keep_links=False)
sh_names = [w for w in wb.sheetnames if w not in ["Team"]]
wb.close()

data = dict()
for sh_name in sh_names:
    df = pd.read_excel(xlsm, sheet_name=sh_name)
    if sh_name=="schedule":
        # Schedule: ongoing project only
        df = df.drop(columns=["start_date","days"]).fillna(0)
        df = df.loc[df["type"].isin(["Project"]) & 
                    (df["completion"]<1)].reset_index(drop=True)
        data[sh_name] = df.to_dict()
    elif sh_name=="PublicHolidays":
        # Public holidays
        data[sh_name] = np.array(df.drop(columns=["Holiday"]), 
                                 dtype="datetime64[D]").ravel()
    else: data[sh_name] = df.to_dict()

plt.ioff()
curdir = os.getcwd()
folder = curdir + "\\progress"
if folder.split("\\")[-1] not in os.listdir():
    os.mkdir(folder)

suffix = datetime.now().strftime("%Y-%m-%d")
subfolder = f"{folder}\\{suffix}"
if subfolder.split("\\")[-1] not in os.listdir(folder):
    os.mkdir(f"{folder}\\{suffix}")
    
ref_date = Timestamp(date.today())
holidays = data["PublicHolidays"]
colors = ["#1B9CFC","#55E6C1","#FC427B","#82589F","#FEA47F"]

# Master schedule
df = pd.DataFrame(data["schedule"])
date = ref_date + np.timedelta64(0,"D")
min_date, max_date = df["start"].min(), df["end"].max()
start_date = max(date - np.timedelta64(15,"D"), min_date)
end_date   = min(date + np.timedelta64(15,"D"), max_date)

X = create_schedule(df.copy(), ref_date, holidays=holidays)
X = X.sort_values(by="start", ascending=False).reset_index(drop=True)
ax = gantt_plot(X, ref_date=ref_date, holidays=holidays, 
                colors=colors, start_date=start_date, end_date=end_date)
legend = ax.get_legend()
legend.set_bbox_to_anchor([1.1,0.8], transform=ax.transAxes)
plt.gcf().suptitle("Master Schedule", fontsize=15)
plt.tight_layout()
plt.savefig(subfolder + f"\\masterschedule_{suffix}.png", dpi=200)

tasks = dict([(a,b) for a,b in zip(data["schedule"]["ref_id"].values(),
                                   data["schedule"]["task"].values())])
for project in data["schedule"]["ref_id"].values():
    if project in data.keys():
        df = pd.DataFrame(data[project])   
        df = create_schedule(df, ref_date, holidays)
        df = df.sort_index(ascending=False).reset_index(drop=True)
        ax = gantt_plot(df, ref_date=ref_date, 
                        holidays=holidays, colors=colors)
        xmin, xmax = ax.get_xlim()
        ax.set_xlim(xmin-0.5, xmax+0.5)
        legend = ax.get_legend()
        legend.set_bbox_to_anchor([1.1,0.8], transform=ax.transAxes)
        plt.gcf().suptitle(f"{project}: {tasks[project]}", fontsize=15)
        plt.tight_layout()
        plt.savefig(subfolder + f"\\{project}_{suffix}.png", dpi=200)

print("Complete...")

Complete...
